In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
base_dir = '/content/gdrive/My Drive/geomhacks_bias/'

In [0]:
import os
import pandas as pd

def split_3(df, sp1=0.7, sp2=0.8, shuffle=True):
  if shuffle:
    df = df.sample(frac=1)
  l_sp1 = int(sp1 * len(df))
  l_sp2 = int(sp2 * len(df))
  print('%d %d %d' % (l_sp1, l_sp2 - l_sp1, len(df) - l_sp2))
  return df[:l_sp1], df[l_sp1:l_sp2], df[l_sp2:]

def load_and_split():
  df_prom = pd.read_csv(os.path.join(base_dir, 'promotional.csv'))
  df_neutral = pd.read_csv(os.path.join(base_dir, 'good.csv'))
  df_prom = pd.DataFrame(df_prom['text'])
  df_prom['label'] = 1
  df_neutral = pd.DataFrame(df_neutral['text'])
  df_neutral['label'] = 0
  df = pd.concat([df_prom, df_neutral])

  df_train, df_val, df_test = split_3(df)
  df_train.to_csv(os.path.join(base_dir, 'train.csv'))
  df_val.to_csv(os.path.join(base_dir, 'val.csv'))
  df_test.to_csv(os.path.join(base_dir, 'test.csv'))

#load_and_split()

In [0]:
df_train = pd.read_csv(os.path.join(base_dir, 'train.csv'))
df_val = pd.read_csv(os.path.join(base_dir, 'val.csv'))
df_test = pd.read_csv(os.path.join(base_dir, 'test.csv'))

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

!pip install unidecode -q

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import re
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tqdm.notebook import tqdm
from unidecode import unidecode
from textblob import TextBlob

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
cachedStopWords = stopwords.words("english")

def process_text(text):
  # join individual posts
  raw = ' '.join(text.split('|||'))
  
  # unicode to ascii
  result = unidecode(raw)

  # remove anything that's not a letter
  result = re.sub(r"[^a-zA-Z]", " ", result)

  # remove any consecutive spaces
  result = re.sub(' +', ' ', result)

  # lowercase
  result = result.lower()

  # remove stop words
  results = ' '.join([w for w in result.split(' ') if w not in cachedStopWords])

  # stem
  result = stemmer.stem(results)

  # lemmatise
  result = ' '.join([lemmatiser.lemmatize(w) for w in result.split(' ')])

  # remove beginning and end spaces
  result = result.strip()

  return result

def process_df(df):
  X = []
  y = []
  for idx, row in tqdm(df.iterrows(), total=len(df)):
    text = row['text']
    processed = process_text(text)

    #print(processed)

    X.append(processed)
    y.append(row['label'])
  return X, y

In [0]:
import pickle

def preprocess_and_dump():
  X_train_txt, y_train = process_df(df_train)
  X_val_txt, y_val = process_df(df_val)

  with open(os.path.join(base_dir, 'preprocessed.pkl'), 'wb') as file_out:
    pickle.dump({'X_train': X_train_txt, 'y_train': y_train, 'X_val':X_val_txt, 'y_val':y_val}, file_out)

#preprocess_and_dump()

In [0]:
X_test, y_test = process_df(df_test)

In [0]:
def load_preprocess_dump():
  with open(os.path.join(base_dir, 'preprocessed.pkl'), 'rb') as file_in:
    data = pickle.load(file_in)  
  return data['X_train'], data['y_train'], data['X_val'], data['y_val']

In [0]:
X_train_txt, y_train, X_val_txt, y_val = load_preprocess_dump()

In [0]:
from keras.preprocessing.text import Tokenizer

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 400

text_data = [str(txt) for txt in X_train_txt]
val_text_data = [str(txt) for txt in X_val_txt]

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(text_data)

Using TensorFlow backend.


In [0]:
test_text_data = [str(txt) for txt in X_test]

In [0]:
X_train = tokenizer.texts_to_sequences(text_data)
X_val = tokenizer.texts_to_sequences(val_text_data)

In [0]:
X_test = tokenizer.texts_to_sequences(test_text_data)

In [0]:
from keras.preprocessing import sequence

X_train = sequence.pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = sequence.pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
X_test = sequence.pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
from gensim.models import KeyedVectors
import glob
import numpy as np

embedding_dir = '/content/gdrive/My Drive/AP_Compsci_Myers_Briggs/gensim'

EMBEDDING_FILES = [os.path.join(embedding_dir, 'crawl-300d-2M.gensim'),
                   os.path.join(embedding_dir, 'glove.840B.300d.gensim')]

def build_matrix(word_index, path): # https://www.kaggle.com/thousandvoices/simple-lstm/code
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_index.vectors.shape[1]))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
            if candidate in embedding_index:
                embedding_matrix[i] = embedding_index[candidate]
                break
    return embedding_matrix

matrices = []

for f in EMBEDDING_FILES:
  print(f)
  matrices.append(build_matrix(tokenizer.word_index, f))

embedding_matrix = np.concatenate(matrices, axis=-1)

/content/gdrive/My Drive/AP_Compsci_Myers_Briggs/gensim/crawl-300d-2M.gensim


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


/content/gdrive/My Drive/AP_Compsci_Myers_Briggs/gensim/glove.840B.300d.gensim


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Bidirectional, SpatialDropout1D, GRU, Concatenate, Input, GlobalAveragePooling1D, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.initializers import Constant

reg = 0

text_in = Input((MAX_SEQUENCE_LENGTH,))

embed = Embedding(MAX_NB_WORDS,
                    embedding_matrix.shape[1],
                    weights=[embedding_matrix[:MAX_NB_WORDS]],
                    trainable=False)(text_in)

embed = SpatialDropout1D(0.5)(embed)

lstm_out = Bidirectional(LSTM(40,
                          kernel_regularizer=l2(reg),
                          recurrent_regularizer=l2(reg),
                          bias_regularizer=l2(reg)))(embed)

#last_state = Concatenate(axis=-1)([gru_h1, gru_h2])

#concat = GlobalAveragePooling1D()(gru_out)

out = Dense(1, activation='sigmoid')(lstm_out)

model = Model(inputs=[text_in], outputs=[out])

model.compile(loss='binary_crossentropy', optimizer=Adam(1e-2), metrics=['accuracy'])

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 400, 600)          12000000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 400, 600)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 80)                205120    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 81        
Total params: 12,205,201
Trainable params: 205,201
Non-trainable params: 12,000,000
_________________________________________________________________


In [0]:
def find_learning_rate():
  try:
    from keras_lr_finder import LRFinder
  except ModuleNotFoundError:
    print('Installing Keras LRFinder...')
    !pip install -q git+https://github.com/surmenok/keras_lr_finder.git
    from keras_lr_finder import LRFinder

  lr_finder = LRFinder(model)
  lr_finder.find(X_train, y_train, start_lr=0.00001, end_lr=100, batch_size=512, epochs=10)

  lr_finder.plot_loss(n_skip_beginning=20, n_skip_end=5)

#find_learning_rate()

In [0]:
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=[X_val, y_val])

Train on 37881 samples, validate on 5411 samples
Epoch 1/10
37881/37881 [==============================] - 109s 3ms/step - loss: 0.2849 - accuracy: 0.8832 - val_loss: 0.1989 - val_accuracy: 0.9272
Epoch 2/10
37881/37881 [==============================] - 113s 3ms/step - loss: 0.1970 - accuracy: 0.9240 - val_loss: 0.1708 - val_accuracy: 0.9340
Epoch 3/10
37881/37881 [==============================] - 111s 3ms/step - loss: 0.1707 - accuracy: 0.9335 - val_loss: 0.1648 - val_accuracy: 0.9386
Epoch 4/10
37881/37881 [==============================] - 114s 3ms/step - loss: 0.1591 - accuracy: 0.9395 - val_loss: 0.1493 - val_accuracy: 0.9412
Epoch 5/10
37881/37881 [==============================] - 107s 3ms/step - loss: 0.1490 - accuracy: 0.9411 - val_loss: 0.1518 - val_accuracy: 0.9425
Epoch 6/10
33280/37881 [=========================>....] - ETA: 9s - loss: 0.1465 - accuracy: 0.9426 

KeyboardInterrupt: ignored

In [0]:
model.save(os.path.join(base_dir, 'model_%.2f_%.2f.h5' % (0.1518, 0.9425)))

In [0]:
with open(os.path.join(base_dir, 'tokenizer_0.1518_0.9425.pkl'), 'wb') as tkn_out:
  pickle.dump(tokenizer, tkn_out)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [0]:
preds = model.predict(X_test, batch_size=512)

In [0]:
y_test

AttributeError: ignored

In [0]:
f1_score(np.array(y_test), preds[:,0].round())

0.9318352059925094

In [0]:
accuracy_score(np.array(y_test), preds[:,0].round())

0.9411492978566149

In [0]:
with open('tokenizer_0.1496.pkl', 'wb') as tkn_out:
  pickle.dump(tokenizer, tkn_out)

In [0]:
model.save('model_0.1496.h5')
from tensorflow.keras.models import load_model
model_tf = load_model('model_0.1496.h5')

In [0]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

tflite_model_path = 'model_0.1496.tflite'

print(tflite_model_path)

with tf.io.gfile.GFile(tflite_model_path, 'wb') as f:
  f.write(tflite_model)